# Workflow Integration

> End-to-end workflow: Finnish texts → Translation → Audio → Anki cards

In [ ]:
#| default_exp workflow

In [ ]:
#| export
from pathlib import Path

from suomi.core import *
from suomi.tsv import *
from suomi.mp3 import *
from suomi.anki import *

## Main Workflow Function

In [ ]:
#| export
def create_cards(
    texts: list[str],
    deck: str,
    tags: str | list[str] = "lang::fi",
    output_dir: str = "output",
    overwrite: bool = False,
) -> None:
    """
    Create Anki cards from Finnish texts (end-to-end workflow).
    
    Steps:
    1. Translate Finnish → English/Japanese (OpenAI API)
    2. Generate audio files (Piper TTS)
    3. Update TSV with file paths
    4. Upload to Anki (AnkiConnect)
    
    Args:
        texts: List of Finnish words/phrases to create cards for
        deck: Anki deck name (e.g., "06::Daily")
        tags: Tags for the cards (string or list), hierarchical format supported
              Examples: "src::daily", "src::class,level::A1", ["src::medical", "urgent"]
              Note: "lang::fi" is always auto-included
        output_dir: Directory for TSV and audio files (default: "output")
        overwrite: If True, delete existing deck and overwrite TSV file
    
    Example:
        >>> create_cards(
        ...     texts=["kissa", "koira"],
        ...     deck="06::Daily",
        ...     tags="src::daily"
        ... )
    """
    
    tsv_path = f"{output_dir}/{deck.replace(":", "_") + ".tsv"}"
    if overwrite:
        if deck in deckNames():
            deleteDeck(deck)
        # TSV will be auto-overwritten by texts2tsv()
    else:
        if Path(tsv_path).exists():
            raise FileExistsError(
                f"TSV file already exists: {tsv_path}\n"
                f"Use overwrite=True to replace it."
            )
        if deck in deckNames():
            raise ValueError(
                f"Deck '{deck}' already exists in Anki.\n"
                f"Use overwrite=True to replace it."
            )
        
    audio_dir = f"{output_dir}/audio"
    images_dir = f"{output_dir}/images"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    texts2tsv(texts, tsv_path, tags=tags)
    mp3s(tsv_path, output_dir=audio_dir)
    update_tsv_media_paths(tsv_path, dirs=[audio_dir, images_dir])
    addnotes(deck, tsv_path)

## Example Usage

In [ ]:
#| eval: false
txtfn = "lääkarissä_kasvot.txt"
# assert(Path(txtfn).exists())
tsvfn = "output/Ahaa!05_Kasvot.tsv"
# Path(tsvfn).unlink()  
dn = "Ahaa!05:Kasvot"
# deleteDecks([dn])

create_cards(
    texts=cattxt(txtfn),
    deck=dn,
    tags="fi::Kasvot,Ahaa!::05::Lääkarissä",
    overwrite=True,
)

cattsv(tsvfn)[0][:2]

[{'Finnish': 'Mulla on kipeät kasvot.',
  'English': 'I have sore facial features.',
  'Japanese': '顔が痛いです。(かおがいたいです。)',
  'mp3_path': 'output/audio/Ahaa!05_Kasvot_00.mp3',
  'img_path': 'output/images/Ahaa!05_Kasvot.jpg',
  'tags': 'lang/fi,fi::Kasvot,Ahaa!::05::Lääkarissä'},
 {'Finnish': 'Mulla lähtee hiukset.',
  'English': "I'm losing my hair.",
  'Japanese': '髪の毛が抜けています。(かみのけがぬけています。)',
  'mp3_path': 'output/audio/Ahaa!05_Kasvot_01.mp3',
  'img_path': 'output/images/Ahaa!05_Kasvot.jpg',
  'tags': 'lang/fi,fi::Kasvot,Ahaa!::05::Lääkarissä'}]

In [ ]:
#| eval: false
ffr(['output'], ['mp3','png','jpg'], f"{(Path(tsvfn).stem)}")[-4:]

['output/audio/Ahaa!05_Kasvot_12.mp3',
 'output/audio/Ahaa!05_Kasvot_13.mp3',
 'output/audio/Ahaa!05_Kasvot_14.mp3',
 'output/images/Ahaa!05_Kasvot.jpg']

In [ ]:
!tree output

output
├── Ahaa!05_Kasvot.tsv
├── audio
│   ├── Ahaa!05_Kasvot_00.mp3
│   ├── Ahaa!05_Kasvot_01.mp3
│   ├── Ahaa!05_Kasvot_02.mp3
│   ├── Ahaa!05_Kasvot_03.mp3
│   ├── Ahaa!05_Kasvot_04.mp3
│   ├── Ahaa!05_Kasvot_05.mp3
│   ├── Ahaa!05_Kasvot_06.mp3
│   ├── Ahaa!05_Kasvot_07.mp3
│   ├── Ahaa!05_Kasvot_08.mp3
│   ├── Ahaa!05_Kasvot_09.mp3
│   ├── Ahaa!05_Kasvot_10.mp3
│   ├── Ahaa!05_Kasvot_11.mp3
│   ├── Ahaa!05_Kasvot_12.mp3
│   ├── Ahaa!05_Kasvot_13.mp3
│   └── Ahaa!05_Kasvot_14.mp3
└── images
    └── Ahaa!05_Kasvot.jpg

3 directories, 17 files


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()